# Computational Theory - Shane Walsh - G00406694

## Task 1: Binary Representations
Create four functions in Python, demonstrating their use with examples and tests.
1. The function rotl(x, n=1) that rotates the bits in a 32-bit unsigned integer to the left n places.

2. The function rotr(x, n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.

3. The function ch(x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

4. The function maj(x, y, z) which takes a majority vote of the bits in x, y, and z.

The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

## 1. The function rotl(x, n=1)

Bitwise left shift is <<
Bitwise right shift is >>

The use of 0xFFFFFFFF with the AND operator is what's known as masking. This ensures 

In [67]:
import numpy as np

# Math.
import math

def rotl(x, n=1):
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF # << shifts the bits to left, >> shifts the bits to right.

x = 0b1100
print(f'rotl(0b1100, 2): {bin(rotl(x, 2))}')
print(f'rotl(0b1100, 3): {bin(rotl(x, 3))}')

rotl(0b1100, 2): 0b110000
rotl(0b1100, 3): 0b1100000


## 2. The function rotr(x, n=1)

In [68]:
def rotr(x, n=1):
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

x = 0b1100

# Test the function.
print(f'rotr(0b1100, 2): {bin(rotr(x, 2))}')

rotr(0b1100, 2): 0b11


## 3. The function ch(x, y, z)

In [69]:
def ch(x, y, z): # choose between x, y, z. 
    return (x & y) ^ (~x & z) # & is bitwise AND, ^ is bitwise XOR, ~ is bitwise NOT. So this returns x AND y XOR NOT x AND z.

x = 0b1100
y = 0b1010
z = 0b1001

print(f'ch(0b1100, 0b1010, 0b1001): {bin(ch(x, y, z))}')

ch(0b1100, 0b1010, 0b1001): 0b1001


## 4. The function maj(x, y, z)

In [70]:
def maj(x, y, z): # Takes a majority vote of x, y, z.
    return (x & y) ^ (x & z) ^ (y & z) # ^ means XOR. So this returns x AND y XOR x AND z XOR y AND z.

x = 0b1100
y = 0b1010
z = 0b1001

x1 = 0b1100
y1 = 0b1010
z1 = 0b1010

# Examples
print(f'maj(0b1100, 0b1010, 0b1001): {bin(maj(x, y, z))}')
print(f'maj(0b1100, 0b1010, 0b1010): {bin(maj(x1, y1, z1))}')

maj(0b1100, 0b1010, 0b1001): 0b1000
maj(0b1100, 0b1010, 0b1010): 0b1010


## Task 2: Hash Functions
The following hash function is from The C Programming Language by Brian Kernighan and Dennis Ritchie.
Convert it to Python, test it, and suggest why the values 31 and 101 are used.

unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}

I'll now convert this hash function from the C Programming language to python. This will require quite a few changes to the code. We don't need to declare variable types. While the hash function in C uses pointer's and null terminator checking, Python makes this much simpler as we can just iterate directly over the string. 

In [71]:
def Hash_string(s): # No need to declare var, just use it with the input, no pointer.
    hashval = 0 # No use of 'unsigned' in Python.
    # Iterate directly over string.
    for c in s:
        hashval = ord(c) + 31 * hashval # ord() returns ASCII value of char, 31 is prime number, good for hashing.
    return hashval % 101 # Modulus is unchanged, 101 is prime number, reduces collisions.

The final conversion would look like this. Rather than dereferencing the character pointer in C, we can use Python's ord() function to pull out the relevant ASCII value of a character. THe modulus element on the hashVal has no necessary changes.

Ord function - https://docs.python.org/3.4/library/functions.html

The numbers 31 and 101 are prime numbers. To my knowledge, using prime numbers for hash values gives us a smoother distribution. 101 helps determine our hash table size and reduce collisions which can quite easily come up. 

In [72]:
# Testing Hash function
print(f'Hash_string("hello"): {Hash_string("hello")}')
print(f'Hash_string("world"): {Hash_string("world")}')
print(f'Hash_string("hello world"): {Hash_string("hello world")}')

Hash_string("hello"): 17
Hash_string("world"): 34
Hash_string("hello world"): 13


## Task 3: SHA256

Write a Python function that calculates the SHA256 padding for a given file.  
The function should take a file path as input.  
It should print, in hex, the padding that would be applied to it.  
The [specification](https://doi.org/10.6028/NIST.FIPS.180-4) states that the following should be appended to a message:  

- a`1` bit;
- enough `0` bits so the length in bits of padded message is the smallest possible multiple of 512;
- the length in bits of the original input as a big-endian 64-bit unsigned integer.

The example in the specification is a file containing the three bytes `abc`:  

```python
01100001 01100010 01100011
```

The output would be:  

```python
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18
```

## Task 4: Prime Numbers

Calculate the first 100 prime numbers using two different algorithms.  
Any algorithms that are well-established and works correctly are okay to use.  
Explain how the algorithms work.